Script to analyze a couple of atomic distances in an MD trajectory
In this case, the atoms are accessed by the numbering in the PDB file, as the calculation
comes from a QMMM run with newchem

JVF 20/04/20

In [ ]:
import os
import MDAnalysis
import numpy as np

orig_path=os.path.expanduser("data/")
xyzfile=orig_path+'opt1.pdb'
trjfile=orig_path+'opt1.dcd'
u = MDAnalysis.Universe(xyzfile,trjfile)

In [ ]:
print(u.atoms.names)

In [ ]:
print(u.trajectory)

In [ ]:
# select three atoms to calculate some distances
at1=u.select_atoms("bynum 12")
at2=u.select_atoms("bynum 11")
at3=u.select_atoms("bynum 14")
print(at1,at2,at3)
print(at1.positions[0])

In [ ]:
import nglview

In [ ]:
view = nglview.show_mdanalysis(u)
view.add_ball_and_stick()
view

In [ ]:
# in case we want to fix the movie in a single frame
#view.frame = 4

In [ ]:
# plot the distance between atoms at1 and at2 and between at2 and at3 defined above
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


#plt.style.use('ggplot')
#sns.set_style('ticks')

# calculate the distances
distance12 = np.zeros(len(u.trajectory))
distance23 = np.zeros(len(u.trajectory))
times = np.zeros(len(u.trajectory))
for frame, ts in enumerate(u.trajectory):
    at1p=at1.positions[0]
    at2p=at2.positions[0]
    at3p=at3.positions[0]
    times[frame] = u.trajectory.time
    distance12[frame] = np.linalg.norm(at1p-at2p)
    distance23[frame] = np.linalg.norm(at2p-at3p)

# set the limits of the Y axis
maxY=max(np.maximum(distance12,distance23))
minY=min(np.minimum(distance12,distance23))
fig=plt.figure()
ax = fig.add_subplot(111)
name1=at1[0].name
name2=at2[0].name
name3=at3[0].name
ax.plot(times, distance12,label="dist "+name1+"-"+name2)
ax.plot(times, distance23,label="dist "+name2+"-"+name3)
sns.despine(ax=ax)
ax.legend(loc="center right")
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"distance ($\AA$)")
ax.set_xlim(0, max(times))
ax.set_ylim(minY, maxY)
fig.savefig("output/distance.pdf")
